In [1]:

!pip install openpyxl PyPDF2 tabula-py -q

import pandas as pd
import PyPDF2
import tabula

print("✅ Libraries installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 111.1 MB/s eta 0:00:00
✅ Libraries installed!


In [2]:
# Loading the enrollment Excel file
enrollment_file = 'enrollment_2425.xlsx'

# Checking what sheets are in the file
xl_file = pd.ExcelFile(enrollment_file)
print("📋 Available sheets:")
for i, sheet in enumerate(xl_file.sheet_names):
    print(f"  {i}: {sheet}")

📋 Available sheets:
  0: Introduction and changes
  1: State
  2: District
  3: School


In [4]:
# Loading the School sheet

df = pd.read_excel(enrollment_file, sheet_name='School', header=2)

print(f"✅ Loaded {len(df)} schools")
print(f"\n📋 Column names:")
for i, col in enumerate(df.columns):
    print(f"  {i}: {col}")

✅ Loaded 2506 schools

📋 Column names:
  0: County Code
  1: County Name
  2: District Code
  3: District Name
  4: School Code
  5: School Name
  6: Total Enrollment
  7: White
  8: %White
  9: Black
  10: %Black
  11: Hispanic
  12: %Hispanic
  13: Asian
  14: %Asian
  15: Native American
  16: %Native American
  17: Hawaiian Native
  18: %Hawaiian Native
  19: Two or More Races
  20: %Two or More Races
  21: Pre-K Halfday
  22: %Pre-K Halfday
  23: Pre-K FullDay
  24: %Pre-K FullDay
  25: Kindergarten Halfday
  26: %Kindergarten Halfday
  27: Kindergarten Fullday
  28: %Kindergarten Fullday
  29: First Grade
  30: %First Grade
  31: Second Grade
  32: %Second Grade
  33: Third Grade
  34: %Third Grade
  35: Fourth Grade
  36: %Fourth Grade
  37: Fifth Grade
  38: %Fifth Grade
  39: Sixth Grade
  40: %Sixth Grade
  41: Seventh Grade
  42: %Seventh Grade
  43: Eighth Grade
  44: %Eighth Grade
  45: Ninth Grade
  46: %Ninth Grade
  47: Tenth Grade
  48: %Tenth Grade
  49: Eleventh Grad

In [5]:
print(df.head())

  County Code County Name  District Code                    District Name  \
0          01    Atlantic           10.0  Absecon Public Schools District   
1          01    Atlantic           10.0  Absecon Public Schools District   
2          01    Atlantic          110.0    Atlantic City School District   
3          01    Atlantic          110.0    Atlantic City School District   
4          01    Atlantic          110.0    Atlantic City School District   

   School Code                School Name  Total Enrollment  White  %White  \
0         50.0             Emma C Attales             392.0  171.0    43.6   
1         60.0             H Ashton Marsh             518.0  227.0    43.8   
2         10.0  Atlantic City High School            1731.0   90.0     5.2   
3        300.0     Brighton Avenue School             302.0    6.0     2.0   
4         50.0     Chelsea Heights School             267.0   10.0     3.7   

   Black  ...  Multilingual Learners  %Multilingual Learners  Migran

In [6]:
# Creating county-level rankings
county_rankings = df_filtered.groupby('County Name').agg({
    'Total_Grade_5_8': 'sum',
    'School Name': 'count'
}).reset_index()

county_rankings.columns = ['County', 'Total_Students_Grade_5_8', 'Number_of_Schools']

# Adding ranking
county_rankings['Rank'] = county_rankings['Total_Students_Grade_5_8'].rank(ascending=False, method='min').astype(int)

# Sorting by rank
county_rankings = county_rankings.sort_values('Rank')

print("🏆 TOP 10 COUNTIES BY GRADES 5-8 ENROLLMENT:\n")
print(county_rankings.head(10).to_string(index=False))

# Saving it to CSV
county_rankings.to_csv('county_rankings.csv', index=False)
print("\n✅ Saved: county_rankings.csv")
print("All columns:")
for i, col in enumerate(df.columns):
    print(f"{i}: {col}")

All columns:
0: County Code
1: County Name
2: District Code
3: District Name
4: School Code
5: School Name
6: Total Enrollment
7: White
8: %White
9: Black
10: %Black
11: Hispanic
12: %Hispanic
13: Asian
14: %Asian
15: Native American
16: %Native American
17: Hawaiian Native
18: %Hawaiian Native
19: Two or More Races
20: %Two or More Races
21: Pre-K Halfday
22: %Pre-K Halfday
23: Pre-K FullDay
24: %Pre-K FullDay
25: Kindergarten Halfday
26: %Kindergarten Halfday
27: Kindergarten Fullday
28: %Kindergarten Fullday
29: First Grade
30: %First Grade
31: Second Grade
32: %Second Grade
33: Third Grade
34: %Third Grade
35: Fourth Grade
36: %Fourth Grade
37: Fifth Grade
38: %Fifth Grade
39: Sixth Grade
40: %Sixth Grade
41: Seventh Grade
42: %Seventh Grade
43: Eighth Grade
44: %Eighth Grade
45: Ninth Grade
46: %Ninth Grade
47: Tenth Grade
48: %Tenth Grade
49: Eleventh Grade
50: %Eleventh Grade
51: Twelfth Grade
52: %Twelfth Grade
53: Free Lunch
54: %Free Lunch
55: Reduced Lunch
56: %Reduced Lunch

In [7]:
# Calculating total students in grades 5-8
df['Total_Grade_5_8'] = (
    df['Fifth Grade'].fillna(0) +
    df['Sixth Grade'].fillna(0) +
    df['Seventh Grade'].fillna(0) +
    df['Eighth Grade'].fillna(0)
)

# Filtering schools that have grades 5-8 students
df_filtered = df[df['Total_Grade_5_8'] > 0].copy()

print(f"✅ Calculated grades 5-8 totals")
print(f"📊 {len(df_filtered)} schools have students in grades 5-8")
print(f"\n👀 Sample data:")
print(df_filtered[['County Name', 'District Name', 'School Name', 'Fifth Grade', 'Sixth Grade', 'Seventh Grade', 'Eighth Grade', 'Total_Grade_5_8']].head(10))

✅ Calculated grades 5-8 totals
📊 1611 schools have students in grades 5-8

👀 Sample data:
   County Name                    District Name  \
0     Atlantic  Absecon Public Schools District   
1     Atlantic  Absecon Public Schools District   
3     Atlantic    Atlantic City School District   
4     Atlantic    Atlantic City School District   
5     Atlantic    Atlantic City School District   
6     Atlantic    Atlantic City School District   
7     Atlantic    Atlantic City School District   
8     Atlantic    Atlantic City School District   
9     Atlantic    Atlantic City School District   
10    Atlantic    Atlantic City School District   

                                School Name  Fifth Grade  Sixth Grade  \
0                            Emma C Attales         88.0         84.0   
1                            H Ashton Marsh          2.0          0.0   
3                    Brighton Avenue School         34.0          0.0   
4                    Chelsea Heights School         32.0

In [8]:
# Creating county-level rankings
county_rankings = df_filtered.groupby('County Name').agg({
    'Total_Grade_5_8': 'sum',
    'School Name': 'count'
}).reset_index()

county_rankings.columns = ['County', 'Total_Students_Grade_5_8', 'Number_of_Schools']

# Adding ranking
county_rankings['Rank'] = county_rankings['Total_Students_Grade_5_8'].rank(ascending=False, method='min').astype(int)

# Sorting by rank
county_rankings = county_rankings.sort_values('Rank')

print("🏆 TOP 10 COUNTIES BY GRADES 5-8 ENROLLMENT:\n")
print(county_rankings.head(10).to_string(index=False))

# Saving it to CSV
county_rankings.to_csv('county_rankings.csv', index=False)
print("\n✅ Saved: county_rankings.csv")

🏆 TOP 10 COUNTIES BY GRADES 5-8 ENROLLMENT:

   County  Total_Students_Grade_5_8  Number_of_Schools  Rank
   Bergen                   39416.0                176     1
Middlesex                   36741.0                118     2
    Essex                   34872.0                154     3
    Union                   27554.0                111     4
 Monmouth                   25894.0                108     5
 Charters                   22960.0                 78     6
   Camden                   22615.0                 97     7
   Hudson                   21737.0                 84     8
   Morris                   21635.0                 99     9
  Passaic                   20904.0                 98    10

✅ Saved: county_rankings.csv


In [9]:
# Adding ranking within each district
df_filtered['Rank_in_District'] = df_filtered.groupby('District Name')['Total_Grade_5_8'].rank(ascending=False, method='min').astype(int)

# Creating school rankings dataframe
school_rankings = df_filtered[[
    'County Name',
    'District Name',
    'School Name',
    'Total_Grade_5_8',
    'Rank_in_District'
]].copy()

# Sorting by district and rank
school_rankings = school_rankings.sort_values(['District Name', 'Rank_in_District'])

print("🏫 SAMPLE: Top 3 Schools in First 5 Districts:\n")
print(school_rankings.groupby('District Name').head(3).head(15).to_string(index=False))

# Saving it to CSV
school_rankings.to_csv('school_rankings_by_district.csv', index=False)
print("\n✅ Saved: school_rankings_by_district.csv")

🏫 SAMPLE: Top 3 Schools in First 5 Districts:

County Name                               District Name                                 School Name  Total_Grade_5_8  Rank_in_District
   Atlantic             Absecon Public Schools District                              Emma C Attales            392.0                 1
   Atlantic             Absecon Public Schools District                              H Ashton Marsh              2.0                 2
   Charters Academy For Urban Leadership Charter School Academy For Urban Leadership Charter School             53.0                 1
   Charters Achievers Early College Prep Charter School Achievers Early College Prep Charter School            275.0                 1
  Hunterdon         Alexandria Township School District                  Alexandria Township School            168.0                 1
     Warren          Allamuchy Township School District                   Allamuchy Township School            165.0                 1
     Ber

In [10]:
# Loading the Title 1 PDF
pdf_file = 'newjerseypdf-40553.pdf'

# Extracting text from PDF
with open(pdf_file, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    print(f"📄 PDF has {len(pdf_reader.pages)} pages\n")

    # Extracting first page to see the structure
    print("📋 First page content:\n")
    text = pdf_reader.pages[0].extract_text()
    print(text[:1500])

📄 PDF has 12 pages

📋 First page content:

Title I Grants to Local Educational Agencies - NEW JERSEY
Allocations under the American Recovery and Reinvestment Act
Title I Allocations
LEA ID District Under the Recovery Act*
3400660 ABSECON CITY 90,622
3400690 ALEXANDRIA TOWNSHIP 03400720 ALLAMUCHY TOWNSHIP 03400750 ALLENDALE BOROUGH 03400780 ALLENHURST 0
3400810 ALLOWAY TOWNSHIP 42,6973400840 ALPHA BOROUGH 44,440
3400870 ALPINE BOROUGH 23,5273400900 ANDOVER REG 0
3400930 ASBURY PARK CITY 1,022,9403400960 ATLANTIC CITY 2,638,3993401020 ATLANTIC HIGHLANDS BOROUGH 03401050 AUDUBON BOROUGH 86,2653401080 AUDUBON PARK BOROUGH 14,2333401110 AVALON BOROUGH 18,2993401140 AVON BOROUGH 0
3416470 BARNEGAT TOWNSHIP 313,6923401170 BARRINGTON BOROUGH 69,7093401200 BASS RIVER TOWNSHIP 03401230 BAY HEAD BOROUGH 8,7143401260 BAYONNE CITY 1,634,6863401290 BEACH HAVEN BOROUGH 10,4563401320 BEDMINSTER TOWNSHIP 03401350 BELLEVILLE TOWN 582,0743401380 BELLMAWR BOROUGH 123,7343401410 BELMAR BOROUGH 61,867340144

In [11]:
# Extracting all Title 1 districts from the PDF
title1_districts = set()

with open(pdf_file, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)

    # Extracting text from all pages
    for page_num in range(len(pdf_reader.pages)):
        text = pdf_reader.pages[page_num].extract_text()
        lines = text.split('\n')

        # Extracting district names (they appear after LEA IDs)
        for line in lines:
            # Looking for lines with district patterns
            # Districts have format: "DISTRICT NAME 03400xxx" or similar
            if len(line) > 10 and any(char.isdigit() for char in line):
                # Extracting the district name part (before the numbers)
                parts = line.split('0340')  # District codes start with 0340
                if len(parts) > 1:
                    district_name = parts[0].strip()
                    if len(district_name) > 5:  # Valid district names
                        title1_districts.add(district_name)

print(f"✅ Found {len(title1_districts)} Title 1 districts\n")
print("📋 Sample Title 1 Districts:")
for i, dist in enumerate(list(title1_districts)[:20]):
    print(f"  {i+1}. {dist}")

# Converting to list for filtering
title1_districts_list = list(title1_districts)

✅ Found 42 Title 1 districts

📋 Sample Title 1 Districts:
  1. 3406930 HASBROUCK HEIGHTS BOROUGH
  2. 3408280 LAUREL SPRINGS BOROUGH 16,27
  3. 3401710 BETHLEHEM TOWNSHIP
  4. 3403420 COLLINGSWOOD BOROUGH 174,2743403450 COLTS NECK TOWNSHIP
  5. 3408010 KINGWOOD TOWNSHIP
  6. 3406750 HARDWICK TOWNSHIP
  7. 3408060 KITTATINNY REGIONAL
  8. 3407110 HIGH BRIDGE BOROUGH
  9. 3403540 CRANBURY TOWNSHIP
  10. 3401770 BLACK HORSE PIKE REGIONAL 306,7223401800 BLAIRSTOWN TOWNSHIP
  11. 3407590 HUNTERDON CENTRAL REG
  12. 3408500 LENAPE VALLEY REGIONAL
  13. 3409840 MEDFORD LAKES BOROUGH
  14. 3405910 GLEN GARDNER BOROUGH
  15. 3407500 HOPEWELL TOWNSHIP 56,6393407530 HOPEWELL VALLEY REGIONAL
  16. 3409360 MAINLAND REGIONAL 94,9793409390 MANALAPAN-ENGLISHTOWN REG
  17. 3402340 BRIGANTINE CITY 275,7563402370 BROOKLAWN BOROUGH 68,5623402400 BUENA REGIONAL 407,21
  18. 3415450 DELSEA REGIONAL HIGH SCHOOL DISTRICT 127,22
  19. 3414610 SANDYSTON-WALPACK TOWNSHIP 10,4563414640 SAYREVILLE BOROUGH
  20. 34

In [15]:
# Title 1 districts list (manually cleaned from PDF)
title1_districts_manual = [
    'ABSECON',
    'ALEXANDRIA',
    'ALLAMUCHY',
    'ALLENDALE',
    'ALLOWAY',
    'ALPHA',
    'ALPINE',
    'ANDOVER',
    'ASBURY PARK',
    'ATLANTIC CITY',
    'BARNEGAT',
    'BARRINGTON',
    'BASS RIVER',
    'BAY HEAD',
    'BAYONNE',
    'BETHLEHEM',
    'BEVERLY',
    'BLACK HORSE PIKE',
    'BLAIRSTOWN',
    'BLOOMFIELD',
    'BLOOMINGDALE',
    'BOONTON',
    'BRIGANTINE',
    'BROOKLAWN',
    'BUENA',
    'COLLINGSWOOD',
    'COLTS NECK',
    'CRANBURY',
    'DELSEA',
    'DUNELLEN',
    'EAGLESWOOD',
    'EAST AMWELL',
    'GLEN GARDNER',
    'HARDWICK',
    'HASBROUCK HEIGHTS',
    'HIGH BRIDGE',
    'HOPEWELL',
    'HUNTERDON CENTRAL',
    'KINGWOOD',
    'KITTATINNY',
    'LAUREL SPRINGS',
    'LENAPE VALLEY',
    'MAINLAND',
    'MANALAPAN',
    'MEDFORD LAKES',
    'SANDYSTON',
    'SAYREVILLE'
]

print(f"✅ Loaded {len(title1_districts_manual)} Title 1 districts")

✅ Loaded 47 Title 1 districts


In [16]:
# Filtering for Title 1 schools
df_filtered['Is_Title1'] = df_filtered['District Name'].apply(
    lambda x: any(t1.upper() in str(x).upper() for t1 in title1_districts_manual)
)

df_title1 = df_filtered[df_filtered['Is_Title1'] == True].copy()

print(f"✅ Found {len(df_title1)} Title 1 schools")
print(f"📊 Total students (grades 5-8): {df_title1['Total_Grade_5_8'].sum():,.0f}")

✅ Found 102 Title 1 schools
📊 Total students (grades 5-8): 22,731


In [17]:
# County rankings for Title 1 schools
county_rankings = df_title1.groupby('County Name').agg({
    'Total_Grade_5_8': 'sum',
    'School Name': 'count'
}).reset_index()

county_rankings.columns = ['County', 'Total_Students_Grade_5_8', 'Number_of_Schools']
county_rankings['Rank'] = county_rankings['Total_Students_Grade_5_8'].rank(ascending=False, method='min').astype(int)
county_rankings = county_rankings.sort_values('Rank')

print("🏆 COUNTY RANKINGS (Title 1 Schools Only):\n")
print(county_rankings.to_string(index=False))

county_rankings.to_csv('county_rankings_title1.csv', index=False)
print("\n✅ Saved: county_rankings_title1.csv")

🏆 COUNTY RANKINGS (Title 1 Schools Only):

    County  Total_Students_Grade_5_8  Number_of_Schools  Rank
  Atlantic                    4202.0                 19     1
    Hudson                    3074.0                 11     2
  Monmouth                    3041.0                 11     3
 Middlesex                    2398.0                  5     4
     Essex                    1843.0                 10     5
     Ocean                    1115.0                  4     6
    Mercer                    1043.0                  5     7
    Bergen                    1039.0                  5     8
 Hunterdon                     988.0                  7     9
    Camden                     969.0                  9    10
Gloucester                     510.0                  1    11
    Morris                     496.0                  2    12
    Sussex                     449.0                  3    13
    Warren                     379.0                  3    14
Burlington                 

In [18]:
# School rankings within districts
df_title1['Rank_in_District'] = df_title1.groupby('District Name')['Total_Grade_5_8'].rank(ascending=False, method='min').astype(int)

school_rankings = df_title1[[
    'County Name', 'District Name', 'School Name',
    'Total_Grade_5_8', 'Rank_in_District'
]].sort_values(['District Name', 'Rank_in_District'])

print("🏫 SCHOOL RANKINGS BY DISTRICT (Sample):\n")
print(school_rankings.head(20).to_string(index=False))

school_rankings.to_csv('school_rankings_title1.csv', index=False)
print("\n✅ Saved: school_rankings_title1.csv")

🏫 SCHOOL RANKINGS BY DISTRICT (Sample):

County Name                       District Name                               School Name  Total_Grade_5_8  Rank_in_District
   Atlantic     Absecon Public Schools District                            Emma C Attales            392.0                 1
   Atlantic     Absecon Public Schools District                            H Ashton Marsh              2.0                 2
  Hunterdon Alexandria Township School District                Alexandria Township School            168.0                 1
     Warren  Allamuchy Township School District                 Allamuchy Township School            165.0                 1
     Bergen    Allendale Public School District                          Brookside School            407.0                 1
      Salem         Alloway Twp School District                   Alloway Township School            124.0                 1
     Warren       Alpha Borough School District                      Alpha Borough S

In [20]:
from google.colab import files

files.download('county_rankings_title1.csv')
files.download('school_rankings_title1.csv')

print("✅ Files downloaded! Next: Google Looker Studio!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Files downloaded! Next: Google Looker Studio!
